### Shift Scheduler

In various companies, we implement on-call duties wherein an employee needs to remain available during designated timeframes and be capable of addressing any arising issues. This practice is typically organized through shifts, where each worker assumes on-call responsibilities for a specific period before being relieved by another colleague. However, the challenge lies in the fact that the distribution of these shifts is often not optimized, leading to potential imbalances where some individuals end up working more frequently than others during critical periods such as holidays or birthdays. This problem can be effectively addressed through the application of a straightforward algorithm, ensuring a fairer allocation of shifts across the team. By achieving a fairer distribution, this approach enhances worker satisfaction, minimizes frequent shift changes — a source of frustration - and thereby contributes to a smoother work environment.


Let's establish the target year for our scheduling and define the default properties for the on-call shifts.


In [155]:
import datetime
import math
import pandas as pd
import os
import json
import random

year = 2024

# Get total number of weeks of the year, starting to count at the first monday
first_monday = datetime.date(
    year, 1, 1) + datetime.timedelta(days=7 - datetime.date(year, 1, 1).weekday())
last_monday = datetime.date(
    year, 12, 31) - datetime.timedelta(days=datetime.date(year, 12, 31).weekday())
total_weeks = math.ceil((last_monday - first_monday).days / 7) + 1


Now let's import the json files filled with team data. Each file must contain each person preferences of restricted days, following the dateformat defined above. SEV2 days should be the most restrictive, followed by SEV3. SEV1 are prohibited days and should only exist in case of OOTOs already scheduled. The file must be named as the member name, and must be in the folder assets.


In [156]:
def convert_to_weeks(days):
    weeks = []
    for day in days:
        date = datetime.datetime.strptime(day + '-' + str(year),
                                          "%d-%m-%Y").date()
        weeks.append((date - first_monday).days // 7)
    return weeks


members = []
columns = ['SEV1', 'SEV2', 'SEV3']
team_preferences = pd.DataFrame(columns=columns)

for file in os.listdir('assets'):
    if file.endswith('.json'):
        member = file.strip('.json')
        members.append(member)

        with open('assets/' + file) as json_file:
            content = json.load(json_file)
            team_preferences.loc[member] = [convert_to_weeks(content['SEV1']),
                                            convert_to_weeks(content['SEV2']),
                                            convert_to_weeks(content['SEV3'])]

print(team_preferences)

min_shifts = total_weeks // len(members)
max_shifts = total_weeks // len(members) + 1


               SEV1      SEV2    SEV3
Amelia     [50, 21]  [50, 21]  [5, 5]
Gabriel          []  [50, 51]    [16]
Fi               []  [48, 48]      []
Benjami     [7, 21]  [50, 51]    [16]
Isabella         []  [50, 51]    [16]
Emma             []  [41, 46]    [-1]
Daniel           []  [26, 16]    [34]
Charlotte      [19]  [50, 51]    [16]


Now we have to iterate over the weeks and the members preferences to create the best possible schedule.

In [157]:
def generate_random_permutation():
    permutation = []
    while (len(permutation) < total_weeks):
        available = members.copy()
        while (len(available) > 0):
            random_index = random.randint(0, len(available)-1)
            member = available[random_index]
            permutation.append(member)
            available.remove(member)
    return permutation[:total_weeks]


def check_permutation(permutation):
    for week in range(total_weeks):
        member = permutation[week]
        if (member in permutation[week - math.ceil(len(members)/2) : week]):
            return False
        blocked_weeks = team_preferences.at[member, 'SEV1']
        if (week in blocked_weeks):
            return False
    return True


def calculate_permutation_penalty(permutation):
    penalty = 0
    for week in range(total_weeks):
        member = permutation[week]
        sev2_weeks = team_preferences.at[member, 'SEV2']
        if (week in sev2_weeks):
            penalty += 2
        sev3_weeks = team_preferences.at[member, 'SEV3']
        if (week in sev3_weeks):
            penalty += 1
    return penalty


def generate_valid_permutations():
    while (True):
        permutation = generate_random_permutation()
        if (check_permutation(permutation)):
            penalty = calculate_permutation_penalty(permutation)
            print(penalty, permutation)


generate_valid_permutations()


3 ['Isabella', 'Gabriel', 'Amelia', 'Charlotte', 'Daniel', 'Benjami', 'Emma', 'Fi', 'Charlotte', 'Daniel', 'Benjami', 'Amelia', 'Isabella', 'Fi', 'Gabriel', 'Emma', 'Benjami', 'Isabella', 'Amelia', 'Fi', 'Emma', 'Charlotte', 'Gabriel', 'Daniel', 'Isabella', 'Benjami', 'Amelia', 'Fi', 'Emma', 'Gabriel', 'Daniel', 'Charlotte', 'Fi', 'Benjami', 'Emma', 'Daniel', 'Charlotte', 'Amelia', 'Isabella', 'Gabriel', 'Emma', 'Charlotte', 'Daniel', 'Amelia', 'Isabella', 'Fi', 'Benjami', 'Gabriel', 'Emma', 'Isabella', 'Charlotte', 'Amelia']
5 ['Charlotte', 'Fi', 'Daniel', 'Isabella', 'Gabriel', 'Emma', 'Benjami', 'Amelia', 'Isabella', 'Charlotte', 'Emma', 'Benjami', 'Gabriel', 'Daniel', 'Amelia', 'Fi', 'Charlotte', 'Isabella', 'Emma', 'Benjami', 'Fi', 'Daniel', 'Gabriel', 'Amelia', 'Charlotte', 'Emma', 'Isabella', 'Fi', 'Gabriel', 'Amelia', 'Daniel', 'Benjami', 'Emma', 'Gabriel', 'Amelia', 'Daniel', 'Charlotte', 'Benjami', 'Fi', 'Isabella', 'Daniel', 'Emma', 'Gabriel', 'Charlotte', 'Isabella', 'Ameli

KeyboardInterrupt: 